In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MiAplicacion").getOrCreate()



### Propiedad Descripción

**event_time** - Hora en la que ocurrió el evento (en UTC).

**event_type** - Solo un tipo de evento: view - cart - remove_from_cart - purchase.

**product_id** - ID de un producto.

**category_id** - ID de la categoría del producto.

**category_code** - Taxonomía de la categoría del producto (nombre en código) si fue posible obtenerla. Usualmente presente para categorías significativas y omitida para diferentes tipos de accesorios.

**brand** - Cadena de texto en minúsculas del nombre de la marca. Puede faltar.

**price** - Precio flotante de un producto. Presente.

**user_id** - ID de usuario permanente

**user_session** - ID de sesión temporal del usuario. Es el mismo para cada sesión del usuario. Cambia cada vez que el usuario regresa a la tienda online después de una pausa larga.

In [2]:
from pyspark.sql import functions as F

In [3]:
import os
csv_files = [os.path.join("archive", f) for f in os.listdir("archive") if f.endswith('.csv')]

df = spark.read.options(header='True', inferSchema='True').csv(csv_files)

In [5]:
df.count()

20692840

In [6]:
df.show()

+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|         event_time|      event_type|product_id|        category_id|category_code|    brand|price|  user_id|        user_session|
+-------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|2019-11-30 21:00:00|remove_from_cart|   5712790|1487580005268456287|         NULL|    f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-11-30 21:00:00|            view|   5764655|1487580005411062629|         NULL|      cnd|29.05|412120092|8adff31e-2051-489...|
|2019-11-30 21:00:02|            cart|      4958|1487580009471148064|         NULL|   runail| 1.19|494077766|c99a50e8-2fac-4c4...|
|2019-11-30 21:00:05|            view|   5848413|1487580007675986893|         NULL|freedecor| 0.79|348405118|722ffea5-73c0-492...|
|2019-11-30 21:00:07|            view|   5824148|1487580005511725929|         NULL|

In [7]:
df.filter(F.col("price") < 0).show()

+-------------------+----------+----------+-------------------+-------------+-----+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|category_code|brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+-------------+-----+------+---------+--------------------+
|2019-12-06 05:44:06|  purchase|   5670257|1487580014042939619|         NULL| NULL|-15.87|410016187|2414f145-f7fa-473...|
|2019-12-08 09:38:35|  purchase|   5670257|1487580014042939619|         NULL| NULL|-15.87|579811230|001d3051-07af-1ce...|
|2019-12-13 05:44:49|  purchase|   5716859|1487580014042939619|         NULL| NULL|-47.62|414087247|6617078d-899a-4b8...|
|2019-12-13 10:48:10|  purchase|   5716857|1487580014042939619|         NULL| NULL|-23.81|584773479|a3f156d9-b7fc-444...|
|2019-12-17 07:37:03|  purchase|   5670257|1487580014042939619|         NULL| NULL|-15.87|584498077|ecbf0f7c-3c10-42e...|
|2019-12-17 07:37:03|  p

In [4]:
df = df.withColumn("price",F.when(F.col("price") < 0, F.abs(F.col("price"))).otherwise(F.col("price")))

In [ ]:
df.filter(F.col("price") < 0).show()

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



Tasa de conversión por producto: ¿Cuántos productos que fueron vistos terminaron en compra?

In [ ]:
conversion_reates = df.groupBy("product_id").agg(
    F.sum(F.when(F.col("event_type") == "purchase",1).otherwise(0)).alias("purchases"),
    F.sum(F.when(F.col("event_type") == "view",1).otherwise(0)).alias("views")
).withColumn("conversion_rate", 
            F.when(F.col("views") > 0, F.col("purchases") / F.col("views")).otherwise(0)
)  

In [ ]:
conversion_reates.show()

+----------+---------+-----+--------------------+
|product_id|purchases|views|     conversion_rate|
+----------+---------+-----+--------------------+
|   5819251|       40|  261|  0.1532567049808429|
|   5657304|       12|  571|0.021015761821366025|
|   5815137|       38|  236| 0.16101694915254236|
|   5809871|     1100| 3868|  0.2843846949327818|
|   5812121|      108|  781|  0.1382842509603073|
|   5834157|      126| 1689| 0.07460035523978685|
|   5908237|       20|  100|                 0.2|
|   5826154|       96|  697| 0.13773314203730272|
|   5670719|       69|  114|  0.6052631578947368|
|   5767927|      299|  814| 0.36732186732186733|
|   5574996|       19| 1290|0.014728682170542635|
|   5900643|       58|  331| 0.17522658610271905|
|   5876926|       94| 2055|0.045742092457420926|
|   5863096|      107|  181|  0.5911602209944752|
|   5713340|       15|  110| 0.13636363636363635|
|   5557784|        8|  154| 0.05194805194805195|
|   5797144|       19|  187| 0.10160427807486631|


Tiempo hasta la compra ¿Cuánto tiempo tarda un usuario en comprar desde su primer evento de vista?

In [ ]:
df_filtered = df.filter(F.col("event_type").isin(["view", "purchase"]))

purchase_time = df_filtered.groupBy("user_session","product_id").agg(
    F.min(F.when(F.col("event_type") == "view",F.col("event_time"))).alias("first_view"),
    F.min(F.when(F.col("event_type") == "purchase",F.col("event_time"))).alias("first_purchase")
).withColumn("time_to_purchase", F.col("first_purchase").cast("long") - F.col("first_view").cast("long"))

In [ ]:
purchase_time.filter(F.col("time_to_purchase").isNotNull()).show()

+--------------------+----------+-------------------+-------------------+----------------+
|        user_session|product_id|         first_view|     first_purchase|time_to_purchase|
+--------------------+----------+-------------------+-------------------+----------------+
|885948cc-afa3-46a...|   5866173|2019-10-20 07:39:48|2019-10-20 04:47:40|          -10328|
|8e64c2f8-0104-44e...|   5900297|2019-12-01 03:31:27|2019-12-01 04:35:08|            3821|
|77e818ab-4d9f-4a1...|      4938|2019-12-01 11:24:41|2019-12-01 11:45:48|            1267|
|00e83d38-8c67-4e5...|   5833332|2019-12-01 11:58:48|2019-12-01 12:17:09|            1101|
|8ed1031c-601e-4c8...|   5819270|2019-12-01 12:29:21|2019-12-01 12:42:24|             783|
|a9f51db0-7030-4d5...|   5699414|2019-12-01 13:01:56|2019-12-01 13:38:06|            2170|
|5895cc10-6951-446...|   5787014|2019-12-01 14:32:50|2019-12-01 14:52:21|            1171|
|f8499285-f42b-437...|   5862936|2019-12-01 14:45:35|2019-12-01 15:31:18|            2743|

Algunas dudas sobre este punto. Porque hay negativos? Que pasa si miro un producto luego de comprarlo?

Frecuencia de compra por usuario ¿Qué tan leales son los usuarios? ¿Compran repetidamente?

In [ ]:
purchase_freq = df.filter(F.col("event_type") == "purchase") \
                    .groupBy("user_id") \
                    .agg(F.count("*").alias("total_purchases"))

In [ ]:
purchase_freq.show()

+---------+---------------+
|  user_id|total_purchases|
+---------+---------------+
|578779619|              9|
|580197384|              8|
|479913443|             63|
|579543348|              9|
|539938733|             22|
|569691461|              7|
|522385131|            100|
|461446900|             19|
|445185747|             13|
|398719446|             10|
|543676273|             33|
|580799959|             13|
|581227067|              8|
|568429315|              6|
|563476291|             20|
|439472531|              6|
|582006763|             29|
|582162065|              2|
|582303660|             12|
|569728881|              3|
+---------+---------------+
only showing top 20 rows



Segmentación por precios
¿Cuáles son los rangos de precio más populares?

In [ ]:
price_distribution = df.filter(F.col("event_type") == "purchase") \
                            .groupBy(F.floor(F.col("price") / 50) * 50) \
                            .agg(F.count("*").alias("count")) \
                            .withColumnRenamed("(floor((price / 50)) * 50)", "price range") \
                            .orderBy("count", ascending = False)   

price_distribution.show() 

+-----------+-------+
|price range|  count|
+-----------+-------+
|          0|1280230|
|         50|   5198|
|        150|    655|
|        100|    646|
|        200|    155|
|        250|    116|
|        300|      7|
+-----------+-------+



Análisis de abandono de carrito ¿Qué productos se agregan al carrito pero no se compran?

In [ ]:
cart_vs_purchase = df.groupBy("product_id"). agg(
    F.sum(F.when(F.col("event_type") == "cart", 1).otherwise(0)).alias("added_to_cart"),
    F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchased")


).withColumn("abandon_rate", (F.col("added_to_cart") - F.col("purchased")) / F.col("added_to_cart"))

cart_vs_purchase.show()

+----------+-------------+---------+------------------+
|product_id|added_to_cart|purchased|      abandon_rate|
+----------+-------------+---------+------------------+
|   5819251|          246|       40|0.8373983739837398|
|   5657304|           83|       12|0.8554216867469879|
|   5815137|          153|       38|0.7516339869281046|
|   5809871|         3989|     1100|0.7242416645775883|
|   5812121|          484|      108|0.7768595041322314|
|   5834157|          660|      126|0.8090909090909091|
|   5908237|           95|       20|0.7894736842105263|
|   5826154|          390|       96|0.7538461538461538|
|   5670719|          225|       69|0.6933333333333334|
|   5767927|          809|      299| 0.630407911001236|
|   5574996|          125|       19|             0.848|
|   5900643|          192|       58|0.6979166666666666|
|   5876926|          509|       94|0.8153241650294696|
|   5863096|          637|      107|0.8320251177394035|
|   5713340|           47|       15|0.6808510638

In [9]:
target_product = 5844305

cart_events = df.select("*").filter("event_type='cart'")

sessions_target = cart_events.select(["user_session"]).filter(F.col("product_id") == target_product)

In [22]:
products_with_target = cart_events.select(["product_id"]).filter(F.col("product_id") != target_product).filter(cart_events["user_session"].isin(sessions_target["user_session"]))

In [25]:
count_with_target = products_with_target.groupBy("product_id").agg(F.count("*"))

In [26]:
count_with_target.show()

+----------+--------+
|product_id|count(1)|
+----------+--------+
|   5657304|      83|
|   5815137|     153|
|   5809871|    3989|
|   5834157|     660|
|   5908237|      95|
|   5826154|     390|
|   5670719|     225|
|   5900643|     192|
|   5574996|     125|
|   5863096|     637|
|   5797144|     123|
|   5885863|    1620|
|   5742957|    2040|
|   5882575|     318|
|   5893679|     474|
|   5814871|     282|
|   5649465|     155|
|   5896615|     425|
|   5868397|      29|
|   5816884|      33|
+----------+--------+
only showing top 20 rows



In [28]:
products_with_target.filter(F.col("product_id") == target_product).show()

+----------+
|product_id|
+----------+
+----------+



In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnull, sum

# Crear una sesión de Spark (si aún no la tienes)
spark = SparkSession.builder.appName("ContarNulos").getOrCreate()

# Suponiendo que tu DataFrame se llama 'df'
# Puedes crearlo a partir de un archivo CSV, Parquet, etc.
# Ejemplo:
# df = spark.read.csv("tu_archivo.csv", header=True, inferSchema=True)

# Obtener la lista de nombres de las columnas
columnas = df.columns

# Crear una lista de expresiones para contar los nulos por columna
expresiones_nulos = [sum(isnull(col(c)).cast("integer")).alias(c) for c in columnas]

# Aplicar las expresiones al DataFrame y seleccionar los resultados
conteo_nulos = df.agg(*expresiones_nulos)

# Mostrar el resultado
conteo_nulos.show()

# Para obtener el resultado como un diccionario (opcional)
resultado_dict = conteo_nulos.collect()[0].asDict()
print(resultado_dict)

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión